In [237]:
import pandas as pd
import numpy as np
import re, pytz, os, requests, sys
from pathlib import Path
from datetime import datetime
import sys
sys.path.append("/workspaces/service-data")

from src.clean import clean_percentage, clean_fiscal_yr, normalize_string, standardize_column_names
from src.load import load_csv
from src.export import export_to_csv
from src.merge import merge_si, merge_ss

base_dir = Path.cwd()
parent_dir = base_dir.parent

In [ ]:
si = pd.read_csv("https://github.com/gcperformance/service-data/releases/latest/download/si.csv", 
                 keep_default_na=False, 
                 na_values='', 
                 delimiter=';',
                 engine='python',
                 skipfooter=2
                 )

snap_si = pd.read_csv("https://github.com/gcperformance/service-data/releases/latest/download/2025-03-01_si.csv", 
                 keep_default_na=False, 
                 na_values='', 
                 delimiter=';',
                 engine='python',
                 skipfooter=2
                 )

compare_dict = {
    'key': 'fy_org_id_service_id',
    'df_base': 'si.csv',
    'df_comp': '2025-03-01_si.csv'
}

df_base = si.copy()
df_comp = snap_si.copy()
key = 'fy_org_id_service_id'

base_keys = df_base[key]
comp_keys = df_comp[key]

df_base.set_index(key, inplace=True)
df_comp.set_index(key, inplace=True)

In [239]:
match_keys = pd.merge(base_keys, comp_keys, how='outer', indicator=True)

common_keys = match_keys[match_keys['_merge']=='both'].iloc[:, 0]
uncommon_keys = match_keys[match_keys['_merge']!='both']
keys_only_in_base = match_keys[match_keys['_merge']=='left_only'].iloc[:, 0]
keys_only_in_comp = match_keys[match_keys['_merge']=='right_only'].iloc[:, 0]

In [240]:
df_base_common = df_base[df_base.index.isin(common_keys)]
df_comp_common = df_comp[df_comp.index.isin(common_keys)]

In [ ]:
var_name = 'field'
#ne = not equal, inverse of eq
diff_wide = df_base_common.fillna(0).ne(df_comp_common.fillna(0)).reset_index()
diff = diff_wide.melt(value_vars = diff_wide.columns, var_name=var_name, id_vars = key)
diff = diff[diff['value']]
diff.set_index([key, var_name], inplace=True)

In [ ]:

df_base_long = df_base_common.reset_index().melt(value_vars = df_base_common.columns, var_name = var_name, id_vars=[key])
df_base_long.set_index([key, var_name], inplace=True)

df_comp_long = df_comp_common.reset_index().melt(value_vars = df_comp_common.columns, var_name = var_name, id_vars=[key])
df_comp_long.set_index([key, var_name], inplace=True)

In [ ]:
diff_result = diff.join(df_base_long, how='inner', rsuffix='_base').join(df_comp_long, how='inner', rsuffix='_comp')
diff_result.drop(columns=['value'], inplace=True)
diff_result.reset_index(inplace=True)

In [246]:
keys_only_in_comp.info()

<class 'pandas.core.series.Series'>
Index: 1 entries, 8254 to 8254
Series name: fy_org_id_service_id
Non-Null Count  Dtype 
--------------  ----- 
1 non-null      object
dtypes: object(1)
memory usage: 16.0+ bytes
